In [37]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader
from sagemaker.pytorch import PyTorch

import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

In [16]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [ ]:
train_uri = S3Uploader.upload("../data/churn.csv", 's3://responsibleai/data/churn.csv')
output_path = 's3://responsibleai/dp/output'

In [18]:
pytorch_estimator = PyTorch(entry_point = '../code/train.py',
                            source_dir = '../code',
                            instance_type='ml.p3.2xlarge',
                            output_path = 's3://responsibleai/dp/output',
                            role = role,
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 100, 
                                               'batch-size': 64, 
                                               'learning-rate': 0.003})

In [20]:
pytorch_estimator.fit({'train': 's3://responsibleai/data/churn.csv'})

2022-03-01 17:23:13 Starting - Starting the training job...
2022-03-01 17:23:39 Starting - Launching requested ML instancesProfilerReport-1646155392: InProgress
.........
2022-03-01 17:24:59 Starting - Preparing the instances for training......
2022-03-01 17:26:03 Downloading - Downloading input data
2022-03-01 17:26:03 Training - Downloading the training image.............................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-01 17:30:56,731 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-03-01 17:30:56,755 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-01 17:30:59,783 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-01 17:31:00,137 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt


In [24]:
model_name = "churn-clarify-model" 

predictor = pytorch_estimator.deploy(initial_instance_count=1, 
                         instance_type='ml.p3.2xlarge',
                         model_name = "churn-clarify-model")

Using already existing model: churn-clarify-model


---------!

In [47]:
predictor.delete_endpoint()